In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import class_weight

In [2]:
df = pd.read_csv('df.csv')

In [3]:
df.drop(columns='Unnamed: 0', inplace=True)

In [117]:
df_dep = df[['departure_delay_categorised','HourlyDryBulbTemperature_x_categorised', 'HourlyPrecipitation_x_categorised', 
             'HourlyStationPressure_x_categorised', 'HourlyVisibility_x_categorised', 'HourlyWindSpeed_x_categorised']]

In [119]:
df_dep.to_csv('df_dep.csv')